#Assignment-4
## CSCI 6517 Recommendar Systems

Name: Chowdhury Mohammad Abdullah
<br>
Banner ID: B01019563

## Q1 Implement RAG based Product Recommendation System

### Q1.a. Import Necessary Libraries

In [1]:
# !pip install pandas numpy faiss-cpu sentence-transformers requests

In [2]:
# for data loading, cleaning
import pandas as pd
import numpy as np
import re

#for indexing
import faiss

# for embedding step
from sentence_transformers import SentenceTransformer
import warnings
import pickle

# Enhanced embeddings creation with true granular caching
import pickle
import os
import glob

# Import libraries for LLM integration and API communication
import requests
import json
from typing import List, Dict, Tuple

warnings.filterwarnings('ignore')

2025-08-01 17:52:40.099834: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1754081560.114378  112133 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1754081560.118638  112133 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1754081560.130156  112133 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754081560.130174  112133 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1754081560.130175  112133 computation_placer.cc:177] computation placer alr

### Q1.b. Data Pre-processing + Preparation: Load, Prepare and Pre-process data. Decide what information you will need for your recommendations.  (10)

#### Data Loading and Initial Exploration
This step is necessary for selecting my necessary information before document preparation.

In [3]:
# load dataset
df = pd.read_csv('flipkart_com-ecommerce_sample.csv')

df.head(5)

,uniq_id,crawl_timestamp,product_url,product_name,product_category_tree,pid,retail_price,discounted_price,image,is_FK_Advantage_product,description,product_rating,overall_rating,brand,product_specifications
0,c2d766ca982eca8304150849735ffef9,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2FF9KEDEFGF,999.0,379.0,"[""http://img5a.flixcart.com/image/short/u/4/a/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
1,7f7036a6d550aaa89d34c77bd39a5e48,2016-03-25 22:59:23 +0000,http://www.flipkart.com/fabhomedecor-fabric-do...,FabHomeDecor Fabric Double Sofa Bed,"[""Furniture >> Living Room Furniture >> Sofa B...",SBEEH3QGU7MFYJFY,32157.0,22646.0,"[""http://img6a.flixcart.com/image/sofa-bed/j/f...",False,FabHomeDecor Fabric Double Sofa Bed (Finish Co...,No rating available,No rating available,FabHomeDecor,"{""product_specification""=>[{""key""=>""Installati..."
2,f449ec65dcbc041b6ae5e6a32717d01b,2016-03-25 22:59:23 +0000,http://www.flipkart.com/aw-bellies/p/itmeh4grg...,AW Bellies,"[""Footwear >> Women's Footwear >> Ballerinas >...",SHOEH4GRSUBJGZXE,999.0,499.0,"[""http://img5a.flixcart.com/image/shoe/7/z/z/r...",False,Key Features of AW Bellies Sandals Wedges Heel...,No rating available,No rating available,AW,"{""product_specification""=>[{""key""=>""Ideal For""..."
3,0973b37acd0c664e3de26e97e5571454,2016-03-25 22:59:23 +0000,http://www.flipkart.com/alisha-solid-women-s-c...,Alisha Solid Women's Cycling Shorts,"[""Clothing >> Women's Clothing >> Lingerie, Sl...",SRTEH2F6HUZMQ6SJ,699.0,267.0,"[""http://img5a.flixcart.com/image/short/6/2/h/...",False,Key Features of Alisha Solid Women's Cycling S...,No rating available,No rating available,Alisha,"{""product_specification""=>[{""key""=>""Number of ..."
4,bc940ea42ee6bef5ac7cea3fb5cfbee7,2016-03-25 22:59:23 +0000,http://www.flipkart.com/sicons-all-purpose-arn...,Sicons All Purpose Arnica Dog Shampoo,"[""Pet Supplies >> Grooming >> Skin & Coat Care...",PSOEH3ZYDMSYARJ5,220.0,210.0,"[""http://img5a.flixcart.com/image/pet-shampoo/...",False,Specifications of Sicons All Purpose Arnica Do...,No rating available,No rating available,Sicons,"{""product_specification""=>[{""key""=>""Pet Type"",..."


In [4]:
print(f'Dataset Shape: {df.shape}')
print(f'Products: {df.shape[0]}')
print(f'Columns/ Attributes: {df.shape[1]}')

Dataset Shape: (20000, 15)
Products: 20000
Columns/ Attributes: 15


In [5]:
# List down the columns to chose which features I would Use
for i, col in enumerate(df.columns):
    print(f'{i+1}. {col}')

1. uniq_id
2. crawl_timestamp
3. product_url
4. product_name
5. product_category_tree
6. pid
7. retail_price
8. discounted_price
9. image
10. is_FK_Advantage_product
11. description
12. product_rating
13. overall_rating
14. brand
15. product_specifications


#### Data cleaning

**Keeping:**
* `product_name` - It resembles the core product identity
* `description` - Useful for rich semantic content
* `product_category_tree` - Hierarchical categorization of product
* `brand` - Brand-based filtering/search is important, soemtime it is as important as `product_name` (nike shoes)
* `discounted_price` - Price-aware recommendations (also this is the eventual price the user will consider when buying a product). However, we could make it more complex by also incorporating the `retail_price` because there is a psychology of users on chasing producst on sales, a common purchase behavior exploited by many shops. To keep things simple, we are avoiding it now.
* `pid` - Unique identifier for tracking

**Dropping:**
* `product_url`, `crawl_timestamp` - No semantic value
* `retail_price` - dropping it for simplicity, could be incorporated for higher order reasoning of users' purchase behavior.
* `is_FK_Advantage_product` - Platform-specific metadata, not useful for now
* `product_rating`, `overall_rating` - Mostly missing, so dropping it
* `product_specifications` - Complex nested JSON, requires separate processing, so dropping it
* `image` - Not using visual features

**Price-Aware Enhancement:** I am also incorporating the following price aware enhancement for products for budget-conscious retrieval. I often use these keywords for searching products so I think this should be useful too.
- **Budget Range:** ₹0-2000 → "budget affordable low-cost"
- **Mid-Range:** ₹2001-10000 → "mid-range moderate value"  
- **Premium Range:** ₹10001+ → "premium high-end luxury"

In [6]:
# Text cleaning functions for consistent preprocessing

def clean_text(text):
    """
    Simple text cleaning function:
    - Handle missing values and 'No rating available'
    - Remove JSON-like brackets and quotes
    - Normalize whitespace
    """
    if pd.isna(text) or text == 'No rating available':
        return ""
    
    text = str(text)
    text = re.sub(r'\[.*?\]', '', text)  # Remove brackets and content
    text = re.sub(r'"', '', text)       # Remove quotes
    text = re.sub(r'\s+', ' ', text)    # Normalize whitespace
    return text.strip()

def extract_category_path(category_tree):
    """
    Extract and flatten category hierarchy:
    - Parse JSON-like category structure
    - Return flattened category path
    - Handle missing categories
    """
    if pd.isna(category_tree):
        return ""
    
    try:
        # Handle JSON-like string format
        if isinstance(category_tree, str) and category_tree.startswith('['):
            category_list = eval(category_tree)
            if category_list and len(category_list) > 0:
                # Flatten: "Clothing >> Men's >> Shirts" becomes "Clothing Men's Shirts"
                return category_list[0].replace(' >> ', ' ')
    except:
        pass
    
    return str(category_tree)

def get_price_category(price):
    """
    Simple price categorization:
    - Budget: ≤ ₹2000
    - Mid-range: ₹2001-10000  
    - Premium: > ₹10000
    """
    if pd.isna(price) or price == 0:
        return "unknown-price"
    elif price <= 2000:
        return "budget"
    elif price <= 10000:
        return "mid-range"
    else:
        return "premium"

In [7]:
# Data Preprocessing and Field Selection
# Clean and preprocess the dataset
print("#### Starting data preprocessing #####")

# Create a copy for cleaning
df_clean = df.copy()

# Step 1: DROP rows with missing essential information (keeping it simple)
print(f"Original dataset size: {len(df_clean)}")
df_clean = df_clean.dropna(subset=['product_name'])  # Must have product name
df_clean = df_clean.dropna(subset=['description'])   # Must have description
df_clean = df_clean.dropna(subset=['discounted_price'])  # Must have price
print(f"After dropping missing data: {len(df_clean)}")

# Step 2: Apply cleaning functions to text fields
df_clean['clean_name'] = df_clean['product_name'].apply(clean_text)
df_clean['clean_description'] = df_clean['description'].apply(clean_text)
df_clean['clean_category'] = df_clean['product_category_tree'].apply(extract_category_path)

# Step 3: Handle brand field (fill missing with 'Unknown')
df_clean['clean_brand'] = df_clean['brand'].fillna('Unknown').apply(clean_text)

# Step 4: Add price categories
df_clean['price_category'] = df_clean['discounted_price'].apply(get_price_category)

# Step 5: Remove products with empty names after cleaning
df_clean = df_clean[df_clean['clean_name'] != '']

print(f"Final cleaned dataset size: {len(df_clean)}")
print(f"Products with valid descriptions: {sum(df_clean['clean_description'] != '')}")
print(f"Products with valid categories: {sum(df_clean['clean_category'] != '')}")
print(f"Products with valid brands: {sum(df_clean['clean_brand'] != '')}")

#### Starting data preprocessing #####
Original dataset size: 20000
After dropping missing data: 19920
Final cleaned dataset size: 19920
Products with valid descriptions: 19920
Products with valid categories: 19920
Products with valid brands: 19920


#### Document Prep

**Embedding Strategy and Rationale**

I implemented a **multi-approach embedding strategy** to capture different aspects of product information and match diverse user query patterns. Each aspect of my approach is expected to target specific user search behaviors and information needs. Each approach will be put to the test later when we will be testing retrival later in this notebook.

1. **Product-Focused  -**  This will incorporate the information of `product_name + brand + key_features + price_category` to capture core product identity with brand association ("Samsung smartphone", "Nike shoes"). Adding the price would also help to provide recomendation in terms of user budget.

2. **Semantic-Rich Embedding  -** Incorporates the `product_name + full_description + category_path + price_category` for understanding  maximum semantic information for complex, descriptive queries of user (e.g., "waterproof camera with good battery life").

3. **Category-Optimized Embedding -** Uses the `category_hierarchy + product_name + price_category + brand` for exploratory searches ("kitchen appliances under 5000", "men's formal wear")".

All the three approaches have price aware enhancements.

Finally, I tried to the combination of above 3 in a union fashion too, which is the **Combined approach**. The combined approach is however, very similar to smenatic-rich embedding. But still I gave it a try to explore different combinations.

In [8]:
# Create different document representations for multi-approach embedding
print("Creating document representations for embedding...")

# Initialize document storage
documents = {
    'product_focused': [],      # name + brand + price_category
    'semantic_rich': [],        # name + description + category + price_category  
    'category_optimized': [],    # category + name + brand + price_category
    'combined': []              # hybrid approach, union of above three: name + brand + description + category+ price_category
}

# Process each product to create document variants
for idx, row in df_clean.iterrows():
    name = row['clean_name']
    desc = row['clean_description']
    category = row['clean_category']
    brand = row['clean_brand']
    price_cat = row['price_category']
    
    # Document 1: Product-focused (core identity + brand + price)
    product_focused_doc = f"{name} {brand} {price_cat}".strip()
    documents['product_focused'].append(product_focused_doc)
    
    # Document 2: Semantic-rich (maximum information)
    semantic_rich_doc = f"{name} {desc} {category} {price_cat}".strip()
    documents['semantic_rich'].append(semantic_rich_doc)
    
    # Document 3: Category-optimized (hierarchy + identity + price)
    category_optimized_doc = f"{category} {name} {brand} {price_cat}".strip()
    documents['category_optimized'].append(category_optimized_doc)

    #Document 4: Combined approach
    combined_doc = f"{name} {brand} {desc} {category} {price_cat}".strip()
    documents['combined'].append(combined_doc)

# Display document statistics
for approach, docs in documents.items():
    print(f"{approach.replace('_', ' ').title()} docs: {len(docs)}")

Creating document representations for embedding...
Product Focused docs: 19920
Semantic Rich docs: 19920
Category Optimized docs: 19920
Combined docs: 19920


In [9]:
# checking max document size (for context restriction in generator stage
document_lenghts = []

for docs in documents['combined']:  # only checking the combined documents is sufficient
     document_lenghts.append(len(docs))

print(max(document_lenghts))

5393


In [10]:
print("\nSample documents from first product:\n")

for approach, docs in documents.items():
    print(f"{approach.replace('_', ' ').title()}: {docs[0]} --- Document Size: {len(docs[0])}\n")


Sample documents from first product:

Product Focused: Alisha Solid Women's Cycling Shorts Alisha budget --- Document Size: 49

Semantic Rich: Alisha Solid Women's Cycling Shorts Key Features of Alisha Solid Women's Cycling Shorts Cotton Lycra Navy, Red, Navy,Specifications of Alisha Solid Women's Cycling Shorts Shorts Details Number of Contents in Sales Package Pack of 3 Fabric Cotton Lycra Type Cycling Shorts General Details Pattern Solid Ideal For Women's Fabric Care Gentle Machine Wash in Lukewarm Water, Do Not Bleach Additional Details Style Code ALTHT_3P_21 In the Box 3 shorts Clothing Women's Clothing Lingerie, Sleep & Swimwear Shorts Alisha Shorts Alisha Solid Women's Cycling Shorts budget --- Document Size: 563

Category Optimized: Clothing Women's Clothing Lingerie, Sleep & Swimwear Shorts Alisha Shorts Alisha Solid Women's Cycling Shorts Alisha Solid Women's Cycling Shorts Alisha budget --- Document Size: 159

Combined: Alisha Solid Women's Cycling Shorts Alisha Key Feature

#### Summarry of Data cleaning, and doc creating

In [11]:
print("=== DATA PREPROCESSING SUMMARY ===")
print(f"Original dataset: {len(df)} products")
print(f"Cleaned dataset: {len(df_clean)} products")
print(f"Retention rate: {(len(df_clean)/len(df)*100):.1f}%")

print("\n=== PRICE DISTRIBUTION ===")
price_dist = df_clean['price_category'].value_counts()
print(price_dist)

print("\n=== SAMPLE CLEANED DATA ===")
sample_idx = 0
sample_product = df_clean.iloc[sample_idx]
print(f"Product Name: {sample_product['clean_name']}")
print(f"Brand: {sample_product['clean_brand']}")
print(f"Category: {sample_product['clean_category']}")
print(f"Price: ₹{sample_product['discounted_price']} ({sample_product['price_category']})")
print(f"Description: {sample_product['clean_description'][:200]}...")

print(f"\n=== DOCUMENT VARIANTS FOR EMBEDDING ===")
print("Total document sets created:", len(documents))
print("Each approach contains", len(documents['product_focused']), "documents")

=== DATA PREPROCESSING SUMMARY ===
Original dataset: 20000 products
Cleaned dataset: 19920 products
Retention rate: 99.6%

=== PRICE DISTRIBUTION ===
price_category
budget       18034
mid-range     1061
premium        825
Name: count, dtype: int64

=== SAMPLE CLEANED DATA ===
Product Name: Alisha Solid Women's Cycling Shorts
Brand: Alisha
Category: Clothing Women's Clothing Lingerie, Sleep & Swimwear Shorts Alisha Shorts Alisha Solid Women's Cycling Shorts
Price: ₹379.0 (budget)
Description: Key Features of Alisha Solid Women's Cycling Shorts Cotton Lycra Navy, Red, Navy,Specifications of Alisha Solid Women's Cycling Shorts Shorts Details Number of Contents in Sales Package Pack of 3 Fabr...

=== DOCUMENT VARIANTS FOR EMBEDDING ===
Total document sets created: 4
Each approach contains 19920 documents


### Q1.c. Implementing Retrieval Engine: Index the documents and retrieval engine. Show intermediate outputs of these portions to check the modules are working. (8)

In [12]:
## Checking the faiss version
print(f"FAISS version: {faiss.__version__}")

FAISS version: 1.11.0


#### Initialize Embedding Model

Using `mpnet-base-v2` for creating the embeddings. 

In [13]:
# Initialize the sentence transformer model for creating embeddings
# all-mpnet-base-v2 for high-quality semantic representations
model = SentenceTransformer('all-mpnet-base-v2')

# Display model information
print(f"Embedding dimension: {model.get_sentence_embedding_dimension()}")
print(f"Max sequence length: {model.max_seq_length}")

# Test the model with a sample text
sample_text = "smartphone with good camera"
sample_embedding = model.encode([sample_text])
print(f"Sample embedding shape: {sample_embedding.shape}")

Embedding dimension: 768
Max sequence length: 384
Sample embedding shape: (1, 768)


#### Create Embeddings for All Document Types

This step takes a lot of time so I did caching to save my progreess.

In [14]:
# Create directory for granular cache files
cache_dir = 'embeddings_cache'
os.makedirs(cache_dir, exist_ok=True)

def get_cache_filename(approach_name):
    """Generate cache filename for specific approach"""
    return os.path.join(cache_dir, f'embeddings_{approach_name}.pkl')

def load_cached_approach(approach_name):
    """Load embeddings for a specific approach if available"""
    cache_file = get_cache_filename(approach_name)
    if os.path.exists(cache_file):
        try:
            with open(cache_file, 'rb') as f:
                return pickle.load(f)
        except:
            return None
    return None

def save_approach_cache(approach_name, embeddings_data):
    """Save embeddings for a specific approach"""
    cache_file = get_cache_filename(approach_name)
    with open(cache_file, 'wb') as f:
        pickle.dump(embeddings_data, f)

print("###### Creating embeddings with granular caching ####")
embeddings = {}

# Process each document approach with individual caching
for approach_name, docs in documents.items():
    print(f"\n--- Processing {approach_name.replace('_', ' ').title()} ---")
    
    # Try to load from individual cache first
    cached_embeddings = load_cached_approach(approach_name)
    
    if cached_embeddings is not None:
        # Verify cached data matches current documents
        if len(cached_embeddings) == len(docs):
            print(f" Found valid cache for {approach_name}")
            print(f" Loading {len(cached_embeddings)} embeddings from cache...")
            embeddings[approach_name] = cached_embeddings
            
            # Display stats for loaded embeddings
            print(f"Embeddings shape: {cached_embeddings.shape}")
            print(f"Sample embedding norm: {np.linalg.norm(cached_embeddings[0]):.4f}")
            continue
        else:
            print(f"⚠ Cache size mismatch for {approach_name}, regenerating...")
    
    # Generate embeddings if no valid cache found
    print(f" Generating embeddings for {approach_name}...")
    print(f"Number of documents: {len(docs)}")
    
    approach_embeddings = model.encode(
        docs, 
        show_progress_bar=True, 
        batch_size=32,
        convert_to_numpy=True
    )
    
    # Store in memory
    embeddings[approach_name] = approach_embeddings
    
    # Save individual approach immediately after completion
    print(f" Saving {approach_name} to individual cache...")
    save_approach_cache(approach_name, approach_embeddings)
    print(f" {approach_name} cached successfully!")
    
    # Display embedding statistics
    print(f"Embeddings shape: {approach_embeddings.shape}")
    print(f"Sample embedding norm: {np.linalg.norm(approach_embeddings[0]):.4f}")
    print(f"Memory usage: ~{approach_embeddings.nbytes / 1024 / 1024:.1f} MB")

print(f"\n=== EMBEDDING CREATION SUMMARY ===")
print(f"Total embedding approaches: {len(embeddings)}")
print(f"Documents per approach: {len(embeddings['product_focused'])}")
print(f"Embedding dimension: {embeddings['product_focused'].shape[1]}")
print("##### All embeddings ready ####")

# Show cache status
print(f"\n Cache files created:")
for approach_name in documents.keys():
    cache_file = get_cache_filename(approach_name)
    if os.path.exists(cache_file):
        size_mb = os.path.getsize(cache_file) / 1024 / 1024
        print(f"  {approach_name}: {size_mb:.1f} MB")
    else:
        print(f"  {approach_name}: Not cached")

###### Creating embeddings with granular caching ####

--- Processing Product Focused ---
 Found valid cache for product_focused
 Loading 19920 embeddings from cache...
Embeddings shape: (19920, 768)
Sample embedding norm: 1.0000

--- Processing Semantic Rich ---
 Found valid cache for semantic_rich
 Loading 19920 embeddings from cache...
Embeddings shape: (19920, 768)
Sample embedding norm: 1.0000

--- Processing Category Optimized ---
 Found valid cache for category_optimized
 Loading 19920 embeddings from cache...
Embeddings shape: (19920, 768)
Sample embedding norm: 1.0000

--- Processing Combined ---
 Found valid cache for combined
 Loading 19920 embeddings from cache...
Embeddings shape: (19920, 768)
Sample embedding norm: 1.0000

=== EMBEDDING CREATION SUMMARY ===
Total embedding approaches: 4
Documents per approach: 19920
Embedding dimension: 768
##### All embeddings ready ####

 Cache files created:
  product_focused: 58.4 MB
  semantic_rich: 58.4 MB
  category_optimized: 58.4

#### Build `FAISS` Indexes

In [15]:
# Build FAISS indexes:  efficient similarity search and clustering of dense vectors

indexes = {}
embedding_dim = embeddings['product_focused'].shape[1]  # Get embedding dimension

# Create separate index for each embedding approach
for approach_name, emb_matrix in embeddings.items():
    print(f"\n--- Building index for {approach_name.replace('_', ' ').title()} ---")
    
    # Step 1: Normalize embeddings for cosine similarity
    # L2 normalization enables inner product to compute cosine similarity
    emb_matrix_normalized = emb_matrix / np.linalg.norm(emb_matrix, axis=1, keepdims=True)
    
    # Step 2: Create FAISS index using Inner Product (for cosine similarity)
    # IndexFlatIP = Flat index with Inner Product similarity
    index = faiss.IndexFlatIP(embedding_dim)
    
    # Step 3: Add normalized embeddings to index
    index.add(emb_matrix_normalized.astype('float32'))  # FAISS requires float32
    
    # Store index
    indexes[approach_name] = index
    
    # Display index statistics
    print(f"Index built successfully!")
    print(f"Index type: {type(index)}")
    print(f"Number of vectors indexed: {index.ntotal}")
    print(f"Index dimension: {index.d}")

print(f"\n=== INDEX BUILDING SUMMARY ===")
print(f"Total indexes created: {len(indexes)}")
print(f"Vectors per index: {indexes['product_focused'].ntotal}")
print(f"Index dimension: {indexes['product_focused'].d}")


--- Building index for Product Focused ---
Index built successfully!
Index type: <class 'faiss.swigfaiss_avx2.IndexFlatIP'>
Number of vectors indexed: 19920
Index dimension: 768

--- Building index for Semantic Rich ---
Index built successfully!
Index type: <class 'faiss.swigfaiss_avx2.IndexFlatIP'>
Number of vectors indexed: 19920
Index dimension: 768

--- Building index for Category Optimized ---
Index built successfully!
Index type: <class 'faiss.swigfaiss_avx2.IndexFlatIP'>
Number of vectors indexed: 19920
Index dimension: 768

--- Building index for Combined ---
Index built successfully!
Index type: <class 'faiss.swigfaiss_avx2.IndexFlatIP'>
Number of vectors indexed: 19920
Index dimension: 768

=== INDEX BUILDING SUMMARY ===
Total indexes created: 4
Vectors per index: 19920
Index dimension: 768


**Comment:** So my indexing module is working properly.

#### Retrieval Function

In [16]:
# Create and test the retrieval function (default approach is selected as combined
def retrieve_products(query, approach='combined', top_k=5):
    """
    Retrieve top-k most similar products for a given query
    
    Args:
        query (str): User search query
        approach (str): Embedding approach to use
        top_k (int): Number of products to retrieve
    
    Returns:
        list: Retrieved products with similarity scores
    """
    print(f"--- Retrieving with {approach.replace('_', ' ').title()} approach ---")
    
    # Step 1: Encode the query using the same model (we can not use two different ones)
    query_embedding = model.encode([query])
    
    # Step 2: Normalize query embedding for cosine similarity
    query_normalized = query_embedding / np.linalg.norm(query_embedding, axis=1, keepdims=True)
    
    # Step 3: Search using appropriate FAISS index
    index = indexes[approach]
    similarities, indices = index.search(query_normalized.astype('float32'), top_k)
    
    # Step 4: Prepare results with product information
    results = []
    for i, (idx, sim) in enumerate(zip(indices[0], similarities[0])):
        product = df_clean.iloc[idx]
        
        result = {
            'rank': i + 1,
            'product_id': product['pid'],
            'name': product['clean_name'],
            'brand': product['clean_brand'],
            'price': product['discounted_price'],
            'price_category': product['price_category'],
            'category': product['clean_category'],
            'description': product['clean_description'],
            'similarity_score': float(sim),
            'document_used': documents[approach][idx]
        }
        results.append(result)
    
    return results

#### Test Retrieval with Sample Queries

Here I also decided to compare the four approaches of embeddings alongside.

In [17]:
# Compare retrieval performance across different embedding approaches
comparison_query = "Viari Bag"

print(f"=== Scneario of Testing: Direct product search using brand name ===")
print(f"Query: '{comparison_query}'")
print(f"{'='*70}")

# Test all three approaches for each of the scenario
approaches_to_test = [key for key in documents.keys()]
# ['product_focused', 'semantic_rich', 'category_optimized', 'combined]

for approach in approaches_to_test:
    print(f"\n--- {approach.replace('_', ' ').title()} Approach ---")
    results = retrieve_products(comparison_query, approach=approach, top_k=3)
    
    for result in results:
        print(f"Rank {result['rank']}: {result['name']} | "
              f"₹{result['price']} | Score: {result['similarity_score']:.4f}")
    print("-" * 50)

=== Scneario of Testing: Direct product search using brand name ===
Query: 'Viari Bag'

--- Product Focused Approach ---
--- Retrieving with Product Focused approach ---
Rank 1: Viari Hand-held Bag | ₹12330.0 | Score: 0.8573
Rank 2: Brandvilla Shoulder Bag | ₹2175.0 | Score: 0.5263
Rank 3: Brandvilla Shoulder Bag | ₹2799.0 | Score: 0.5263
--------------------------------------------------

--- Semantic Rich Approach ---
--- Retrieving with Semantic Rich approach ---
Rank 1: Viari Hand-held Bag | ₹12330.0 | Score: 0.6890
Rank 2: Alessia74 Hand-held Bag | ₹649.0 | Score: 0.6065
Rank 3: Fashiondrobe Shoulder Bag | ₹175.0 | Score: 0.5881
--------------------------------------------------

--- Category Optimized Approach ---
--- Retrieving with Category Optimized approach ---
Rank 1: Viari Hand-held Bag | ₹12330.0 | Score: 0.6589
Rank 2: Shagun Trends Pouch Potli | ₹2190.0 | Score: 0.5033
Rank 3: Histeria Hand-held Bag | ₹799.0 | Score: 0.4978
-----------------------------------------------

In [18]:
comparison_query = "waterproof camera with good battery life for camping"

print(f"=== Scneario of Testing: Feature-based searching of products ===")
print(f"Query: '{comparison_query}'")
print(f"{'='*70}")

for approach in approaches_to_test:
    print(f"\n--- {approach.replace('_', ' ').title()} Approach ---")
    results = retrieve_products(comparison_query, approach=approach, top_k=3)
    
    for result in results:
        print(f"Rank {result['rank']}: {result['name']} | "
              f"₹{result['price']} | Score: {result['similarity_score']:.4f}")
    print("-" * 50)

=== Scneario of Testing: Feature-based searching of products ===
Query: 'waterproof camera with good battery life for camping'

--- Product Focused Approach ---
--- Retrieving with Product Focused approach ---
Rank 1: Clicktin Waterproof Backpack | ₹498.0 | Score: 0.5911
Rank 2: Oriflame Waterproof Multipurpose Bag | ₹1599.0 | Score: 0.4815
Rank 3: President School Waterproof Backpack | ₹799.0 | Score: 0.4615
--------------------------------------------------

--- Semantic Rich Approach ---
--- Retrieving with Semantic Rich approach ---
Rank 1: Clicktin Waterproof Backpack | ₹498.0 | Score: 0.4812
Rank 2: President School Waterproof Backpack | ₹785.0 | Score: 0.3977
Rank 3: President School Waterproof Backpack | ₹699.0 | Score: 0.3940
--------------------------------------------------

--- Category Optimized Approach ---
--- Retrieving with Category Optimized approach ---
Rank 1: Uniross HYB 210 9V BT Ready to Use Camera Battery Charger | ₹444.0 | Score: 0.4521
Rank 2: Clicktin Waterpr

### Q1.d. Implementing Generation: Implement generation using a LLAMA3 model. (8)

I used locally running ollama for it.

#### Ollama Client

In [19]:
# Create Ollama client class for local LLAMA3 model interaction
class OllamaClient:
    """
    Client for interacting with locally running Ollama LLAMA3 model
    Handles API communication and response generation
    """
    
    def __init__(self, base_url="http://localhost:11434", model_name="llama3"):
        self.base_url = base_url
        self.model_name = model_name
        print(f"Ollama client initialized for {model_name}")
    
    def test_connection(self):
        """Test connection to Ollama server and verify model availability"""
        try:
            # Check if Ollama is running
            response = requests.get(f"{self.base_url}/api/tags", timeout=10)
            
            if response.status_code == 200:
                models = response.json().get('models', [])
                model_names = [model['name'] for model in models]
                
                print(" Ollama connection successful!")
                print(f"Available models: {model_names}")
                
                # Check if our model is available
                if any(self.model_name in name for name in model_names):
                    print(f" {self.model_name} model found!")
                    return True
                else:
                    print(f"⚠ {self.model_name} model not found in available models")
                    return False
            else:
                print(f"XXXXXX Connection failed: HTTP {response.status_code} XXXXXXX")
                return False
                
        except Exception as e:
            print(f"XXXXX Connection error: {e} XXXXX")
            print("Make sure Ollama is running with: ollama serve")
            return False
    
    def generate(self, prompt, temperature=0.7, top_p=0.9, max_tokens=500):
        """
        Generate text response using LLAMA3 model
        
        Args:
            prompt (str): Input prompt for generation
            temperature (float): Controls randomness (0.0-1.0)
            top_p (float): Nucleus sampling parameter
            max_tokens (int): Maximum tokens to generate
        """
        try:
            # Prepare request payload
            payload = {
                "model": self.model_name,
                "prompt": prompt,
                "stream": False,  # Get complete response
                "options": {
                    "temperature": temperature,
                    "top_p": top_p,
                    "num_predict": max_tokens
                }
            }
            
            # Send request to Ollama
            response = requests.post(
                f"{self.base_url}/api/generate", 
                json=payload,
                timeout=60  # Allow time for generation
            )
            
            if response.status_code == 200:
                result = response.json()
                generated_text = result.get('response', 'No response generated')
                return generated_text
            else:
                return f"Generation error: HTTP {response.status_code}"
                
        except Exception as e:
            return f"Generation error: {e}"

# Initialize and test the client
print("Setting up LLAMA3 client...")
llama_client = OllamaClient()
connection_success = llama_client.test_connection()

Setting up LLAMA3 client...
Ollama client initialized for llama3
 Ollama connection successful!
Available models: ['llama3:latest']
 llama3 model found!


#### RAG Generation Function

In [20]:
# Define default prompt template for RAG generation
DEFAULT_PROMPT = """Your persona: Suppose you are a helpful salesman in the online shop who recommends users what fits their needs best.

User Query: "{query}"

Retrieved Products:
{products_context}

Instructions:
1. For the given list, recommend the most relevant products to customer
2. Explain why each product matches the user's query
3. Mention key features, price, and value proposition
4. If products don't match well, say "Limited relevant options found"
5. Keep recommendations concise and helpful

Provide your recommendations:"""

def generate(query, approach='combined', top_k=3, temperature=0.7, prompt=DEFAULT_PROMPT):
    """
    Complete RAG pipeline: Retrieve relevant products and generate recommendations
    
    Args:
        query (str): User search query
        approach (str): Embedding approach for retrieval
        top_k (int): Number of products to retrieve
        temperature (float): Generation creativity level
        prompt (str): Custom prompt template for generation
    """
    
    print(f"#### Starting RAG generation for: '{query}'")
    
    # Step 1: Retrieve relevant products using our retrieval engine
    print("Step 1: Retrieving relevant products...")
    retrieved_products = retrieve_products(query, approach, top_k)
    
    # Step 2: Prepare context from retrieved products
    print("Step 2: Preparing context for generation...")
    products_context = ""
    for i, product in enumerate(retrieved_products, 1):
        products_context += f"""
Product {i}:
- Name: {product['name']}
- Brand: {product['brand']}
- Price: ₹{product['price']} ({product['price_category']})
- Category: {product['category']}
- Description: {product['description']}
- Relevance Score: {product['similarity_score']:.3f}
"""
    
    # Step 3: Create structured prompt for LLAMA3
    print("Step 3: Creating generation prompt...")
    prompt = prompt.format(
        query=query,
        products_context=products_context
    )
    
    # Step 4: Generate response using LLAMA3
    print("#### Step 4: Generating recommendations with LLAMA3 #####")
    if connection_success:
        response = llama_client.generate(prompt, temperature=temperature)
        return response, retrieved_products
    else:
        return "LLAMA3 model not available. Please ensure Ollama is running with llama3:8b model.", retrieved_products

#### Test RAG Generation

In [21]:
# Test the complete RAG system with sample queries
test_queries = [
    "wireless, waterproof bluetooth headphones for gym and sports",
    "kitchen utensiles for new family"
]

print("=== TESTING COMPLETE RAG SYSTEM ===")

for i, query in enumerate(test_queries, 1):
    print(f"\n{'='*70}")
    print(f"TEST {i}: RAG Generation")
    print(f"{'='*70}")
    print(f"Query: '{query}'")
    print("-" * 50)
    
    # Generate recommendations using RAG
    response, products = generate(
        query, 
        approach='combined',  # considering the most rich embedding approach
        top_k=3,
        temperature=0.7
    )
    
    print("GENERATED RECOMMENDATION:")
    print(response)
    print("\n" + "="*50)

=== TESTING COMPLETE RAG SYSTEM ===

TEST 1: RAG Generation
Query: 'wireless, waterproof bluetooth headphones for gym and sports'
--------------------------------------------------
#### Starting RAG generation for: 'wireless, waterproof bluetooth headphones for gym and sports'
Step 1: Retrieving relevant products...
--- Retrieving with Combined approach ---
Step 2: Preparing context for generation...
Step 3: Creating generation prompt...
#### Step 4: Generating recommendations with LLAMA3 #####
GENERATED RECOMMENDATION:
Based on the user's query for wireless, waterproof Bluetooth headphones for gym and sports, I recommend the following top 2 products:

**Recommended Product 1:** snjmart Gold Note 4 Stereo Dynamic Earphone Wired Headphones (₹299.0)

This product matches the user's query because it's a wired headphone that's sweatproof, making it suitable for gym and sports activities. Although it's not wireless or waterproof Bluetooth headphones as per the exact requirement, its feature

#### Compare Different Generation Settings

The temparature should be checked to get a good balance of correctness and creativeness of our system.

In [22]:
# Test different generation parameters to show system flexibility
test_query = "headphones for musicians"

print(f"=== GENERATION PARAMETER COMPARISON ===")
print(f"Query: '{test_query}'")

# Test different temperature settings
temperatures = [0.3, 0.7, 1.0]
print(f"\n{'='*60}")

for temp in temperatures:
    print(f"\n--- Temperature: {temp} ---")
    print(f"(Lower = more focused, Higher = more creative)")
    
    response, _ = generate(
        test_query, 
        approach='combined',
        top_k=3,
        temperature=temp
    )
    
    # Show first 200 characters to compare styles
    print(f"Response preview: {response}...")
    print("-" * 40)

=== GENERATION PARAMETER COMPARISON ===
Query: 'headphones for musicians'


--- Temperature: 0.3 ---
(Lower = more focused, Higher = more creative)
#### Starting RAG generation for: 'headphones for musicians'
Step 1: Retrieving relevant products...
--- Retrieving with Combined approach ---
Step 2: Preparing context for generation...
Step 3: Creating generation prompt...
#### Step 4: Generating recommendations with LLAMA3 #####
Response preview: Based on the user's query for "headphones for musicians", I recommend the following top two products:

**Recommendation 1:** snjmart For X Play Stereo Dynamic Earphone Wired Headphones (Product 1)

This product matches the user's query well due to its features and specifications. As a musician, you'll appreciate the big speakers that provide amazing sound output, sweat-proof design for intense practice sessions, and a long wire with MIC for hands-free calls. The price of ₹325 is also very budget-friendly.

**Recommendation 2:** snjmart E5-J1 Ace

**Comment:** It seems temprature `0.7` is giving us a proper balance. `0.3` makes the response too rigid, `1.0` makes the generation too lenghty.

#### Test Our Embedding Approaches with Generation

We now check how our four approcahes perform.

In [23]:
# Demonstrate how different retrieval approaches affect generation
test_query = "rugs for kitchen decor"

print(f"=== RETRIEVAL APPROACH IMPACT ON GENERATION ===")
print(f"Query: '{test_query}'")

approaches = ['product_focused', 'semantic_rich', 'category_optimized', 'combined']

for approach in approaches:
    print(f"\n{'='*60}")
    print(f"Using {approach.replace('_', ' ').title()} Retrieval")
    print(f"{'='*60}")
    
    response, products = generate(
        test_query,
        approach=approach,
        top_k=3,
        temperature=0.7
    )
    
    print("GENERATED RECOMMENDATION: ")
    print(response)
    print("\n" + "-"*50)

=== RETRIEVAL APPROACH IMPACT ON GENERATION ===
Query: 'rugs for kitchen decor'

Using Product Focused Retrieval
#### Starting RAG generation for: 'rugs for kitchen decor'
Step 1: Retrieving relevant products...
--- Retrieving with Product Focused approach ---
Step 2: Preparing context for generation...
Step 3: Creating generation prompt...
#### Step 4: Generating recommendations with LLAMA3 #####
GENERATED RECOMMENDATION: 
Based on the query "rugs for kitchen decor", I'd like to recommend the following top two products:

**Recommendation 1:** M S Rugs Pink, Blue Wool Dhurrie (Product 1)

This product is highly relevant to the user's query as it is specifically designed for kitchen decor. The geometric pattern and wool material make it a great match for a kitchen rug. With a price of ₹3100.0, this product offers good value considering its high-quality materials and unique design.

**Recommendation 2:** M S Rugs Pink, Blue Wool Dhurrie (Product 2)

This product is similar to the first r

**Comment:** Combined embedding approach seems to be better working for the queries. It generalizes better than other approach. Semenatically rich approach is also close to it. Product focused approach works good (generates higher cosine simialrity) when user directly asks using code product identity.

### Q1.e. (e) Output and Demonstration: Design prompt(s) and query. Show at least 3 output demonstrations and top-3 relevant products for each demonstration. (5)

#### Design Prompt(s)

I already gave a prompt which indeed has a good balance of context lenght and good generation. Earlier propmts that I used are added here for refrence.

In [24]:
# Define specialized prompts for different query types and scenarios

# Prompt 1: Budget-focused with feature requirements
BUDGET_FEATURE_PROMPT = """You are a budget-conscious shopping advisor helping customers find the best value products.

User Query: "{query}"

Retrieved Products:
{products_context}

Your Task:
1. Focus on products that meet the specific feature requirement AND budget constraint
2. Highlight the value proposition - what makes each product worth the price
3. Compare prices and explain which offers the best deal
4. Mention if any product exceeds budget and suggest alternatives
5. Be honest if no products truly meet both criteria

Provide practical, money-saving recommendations:"""

# Prompt 2: Feature-specific with quality focus  
FEATURE_QUALITY_PROMPT = """You are a tech-savvy product specialist helping customers find high-quality products with specific features.

User Query: "{query}"

Retrieved Products:
{products_context}

Your Expertise:
1. Prioritize products that excel in the requested feature (ignore budget options that compromise quality)
2. Explain technical aspects and why the feature matters
3. Rank products by feature performance, not just price
4. Mention any trade-offs or additional benefits
5. Recommend based on long-term satisfaction rather than immediate savings

Provide expert, feature-focused recommendations:"""

# Prompt 3: Aesthetic and category exploration
AESTHETIC_EXPLORATION_PROMPT = """You are an interior design consultant helping customers create beautiful living spaces.

User Query: "{query}"

Retrieved Products:
{products_context}

Your Design Approach:
1. Focus on aesthetic appeal and how products complement the specified style/colors
2. Suggest how each item would fit into the living space
3. Consider visual harmony and decorative impact
4. Mention styling tips or complementary items if relevant
5. Help create a cohesive look rather than individual product recommendations

Provide creative, style-focused recommendations:"""

# Dictionary to map queries to appropriate prompts
QUERY_PROMPT_MAPPING = {
    'waterproof boots for rain low budget': BUDGET_FEATURE_PROMPT,
    'mid to premium ranged smartphone with long batterylife': FEATURE_QUALITY_PROMPT, 
    'home decor for livingroom blue and white': AESTHETIC_EXPLORATION_PROMPT
}

#### Design Queries

I focused different queries here 

In [25]:
# Define 3 diverse queries covering different product categories and search patterns
demonstration_queries = [
    {
        'query': 'waterproof boots for rain low budget',
        'category': 'Shoes',
        'search_type': 'Budget-focused with feature requirement',
        'expected_focus': 'waterproof + price constraint'
    },
    {
        'query': 'mid to premium ranged smartphone with long batterylife',
        'category': 'Electrincs',
        'search_type': 'Category-specific with specific requirement',
        'expected_focus': 'long battery life, no low end phones'
    },
    {
        'query': 'home decor for livingroom blue and white',
        'category': 'Home & Kitchen',
        'search_type': 'Broad category exploration',
        'expected_focus': 'Color, decor products'
    }
]

#### Demo 1: Budget focused

In [26]:
# === DEMONSTRATION 1: Budget-Focused with Feature Requirements ===
print("=" * 80)
print("DEMONSTRATION 1: Budget-Focused Shopping")
print("=" * 80)

demo1_info = demonstration_queries[0]
query1 = demo1_info['query']

print(f"Query: '{query1}'")
print(f"Category: {demo1_info['category']}")
print(f"Search Type: {demo1_info['search_type']}")
print(f"Expected Focus: {demo1_info['expected_focus']}")
print("\n" + "=" * 50)

# Show the specialized prompt being used
print("PROMPT DESIGN - Budget-Feature Focused:")
print("-" * 40)
print("Persona: Budget-conscious shopping advisor")
print("Focus: Value proposition, price comparison, honest budget assessment")
print("Goal: Find best deals that meet feature requirements")

print("\n" + "=" * 50)
print("RAG GENERATION RESULTS:")
print("=" * 50)

# Generate recommendations using budget-focused prompt
response1, products1 = generate(
    query1, 
    approach='semantic_rich',
    top_k=3,
    prompt=BUDGET_FEATURE_PROMPT
)

print("\nGENERATED RECOMMENDATION:")
print("-" * 30)
print(response1)

print("\n" + "=" * 50)
print("TOP-3 RETRIEVED PRODUCTS:")
print("=" * 50)
for i, product in enumerate(products1, 1):
    print(f"Rank {i}: {product['name']}")
    print(f"  Brand: {product['brand']} | Price: ₹{product['price']} ({product['price_category']})")
    print(f"  Category: {product['category']}")
    print(f"  Similarity Score: {product['similarity_score']:.4f}")
    print()

DEMONSTRATION 1: Budget-Focused Shopping
Query: 'waterproof boots for rain low budget'
Category: Shoes
Search Type: Budget-focused with feature requirement
Expected Focus: waterproof + price constraint

PROMPT DESIGN - Budget-Feature Focused:
----------------------------------------
Persona: Budget-conscious shopping advisor
Focus: Value proposition, price comparison, honest budget assessment
Goal: Find best deals that meet feature requirements

RAG GENERATION RESULTS:
#### Starting RAG generation for: 'waterproof boots for rain low budget'
Step 1: Retrieving relevant products...
--- Retrieving with Semantic Rich approach ---
Step 2: Preparing context for generation...
Step 3: Creating generation prompt...
#### Step 4: Generating recommendations with LLAMA3 #####

GENERATED RECOMMENDATION:
------------------------------
Based on your query for waterproof boots within a low budget of ₹498, I've analyzed the retrieved products and provided my expert advice.

**Recommendation 1:** Product

#### Dem 2: Feature focused

In [27]:
# === DEMONSTRATION 2: Feature-Quality Focused ===
print("=" * 80)
print("DEMONSTRATION 2: Feature-Quality Focused Shopping")
print("=" * 80)

demo2_info = demonstration_queries[1]
query2 = demo2_info['query']

print(f"Query: '{query2}'")
print(f"Category: {demo2_info['category']}")
print(f"Search Type: {demo2_info['search_type']}")
print(f"Expected Focus: {demo2_info['expected_focus']}")
print("\n" + "=" * 50)

# Show the specialized prompt being used
print("PROMPT DESIGN - Feature-Quality Focused:")
print("-" * 40)
print("Persona: Tech-savvy product specialist")
print("Focus: Feature performance, technical expertise, quality over price")
print("Goal: Find products that excel in specific features")

print("\n" + "=" * 50)
print("RAG GENERATION RESULTS:")
print("=" * 50)

# Generate recommendations using feature-quality prompt
response2, products2 = generate(
    query2,
    approach='semantic_rich', 
    top_k=3,
    prompt=FEATURE_QUALITY_PROMPT
)

print("\nGENERATED RECOMMENDATION:")
print("-" * 30)
print(response2)

print("\n" + "=" * 50)
print("TOP-3 RETRIEVED PRODUCTS:")
print("=" * 50)
for i, product in enumerate(products2, 1):
    print(f"Rank {i}: {product['name']}")
    print(f"  Brand: {product['brand']} | Price: ₹{product['price']} ({product['price_category']})")
    print(f"  Category: {product['category']}")
    print(f"  Similarity Score: {product['similarity_score']:.4f}")
    print()

DEMONSTRATION 2: Feature-Quality Focused Shopping
Query: 'mid to premium ranged smartphone with long batterylife'
Category: Electrincs
Search Type: Category-specific with specific requirement
Expected Focus: long battery life, no low end phones

PROMPT DESIGN - Feature-Quality Focused:
----------------------------------------
Persona: Tech-savvy product specialist
Focus: Feature performance, technical expertise, quality over price
Goal: Find products that excel in specific features

RAG GENERATION RESULTS:
#### Starting RAG generation for: 'mid to premium ranged smartphone with long batterylife'
Step 1: Retrieving relevant products...
--- Retrieving with Semantic Rich approach ---
Step 2: Preparing context for generation...
Step 3: Creating generation prompt...
#### Step 4: Generating recommendations with LLAMA3 #####

GENERATED RECOMMENDATION:
------------------------------
Based on your query for a mid-to-premium ranged smartphone with long battery life, I've analyzed the products an

#### Demo 3: Aesthetic-style focused

In [28]:
# === DEMONSTRATION 3: Aesthetic-Style Focused ===
print("=" * 80)
print("DEMONSTRATION 3: Aesthetic-Style Focused Shopping")
print("=" * 80)

demo3_info = demonstration_queries[2]
query3 = demo3_info['query']

print(f"Query: '{query3}'")
print(f"Category: {demo3_info['category']}")
print(f"Search Type: {demo3_info['search_type']}")
print(f"Expected Focus: {demo3_info['expected_focus']}")
print("\n" + "=" * 50)

# Show the specialized prompt being used
print("PROMPT DESIGN - Aesthetic-Exploration Focused:")
print("-" * 40)
print("Persona: Interior design consultant")
print("Focus: Visual appeal, style harmony, decorative impact")
print("Goal: Create cohesive aesthetic recommendations")

print("\n" + "=" * 50)
print("RAG GENERATION RESULTS:")
print("=" * 50)

# Generate recommendations using aesthetic-exploration prompt
response3, products3 = generate(
    query3,
    approach='semantic_rich',
    top_k=3, 
    prompt=AESTHETIC_EXPLORATION_PROMPT
)

print("\nGENERATED RECOMMENDATION:")
print("-" * 30)
print(response3)

print("\n" + "=" * 50)
print("TOP-3 RETRIEVED PRODUCTS:")
print("=" * 50)
for i, product in enumerate(products3, 1):
    print(f"Rank {i}: {product['name']}")
    print(f"  Brand: {product['brand']} | Price: ₹{product['price']} ({product['price_category']})")
    print(f"  Category: {product['category']}")
    print(f"  Similarity Score: {product['similarity_score']:.4f}")
    print()

DEMONSTRATION 3: Aesthetic-Style Focused Shopping
Query: 'home decor for livingroom blue and white'
Category: Home & Kitchen
Search Type: Broad category exploration
Expected Focus: Color, decor products

PROMPT DESIGN - Aesthetic-Exploration Focused:
----------------------------------------
Persona: Interior design consultant
Focus: Visual appeal, style harmony, decorative impact
Goal: Create cohesive aesthetic recommendations

RAG GENERATION RESULTS:
#### Starting RAG generation for: 'home decor for livingroom blue and white'
Step 1: Retrieving relevant products...
--- Retrieving with Semantic Rich approach ---
Step 2: Preparing context for generation...
Step 3: Creating generation prompt...
#### Step 4: Generating recommendations with LLAMA3 #####

GENERATED RECOMMENDATION:
------------------------------
I love working with blue and white themes! Based on your query, I'll provide some design suggestions that incorporate the specified colors while considering aesthetic appeal, visual 

**Comment:** We can see that our system can demonstrate has a very high level of flexibility and we can instruct it to behave only buy using different prompts. This will be useful for scnearios where we know what types of user generally shop in a ecomerce website. For example, if we know that the online shop is premium and customers do have a aesthetic taste, we can simply use the aesthetic prompts and it will align our system's output to the bussiness need. We also see that the generator also gives additional suggestion to user query which may elicit them to provide new querries for co-purchasing new items as well!

## Q2 Answer the following questions

### Q2.a. In Data processing, which information did you retain and which ones you dropped? Why? (2)

I kept the fields that provide comprehensive product information for embedding creation (name, description, category for semantic search) while supporting practical user needs (brand preferences, price constraints). The dropped fields either lacked semantic value, contained incomplete data, or required specialized processing beyond the assignment scope.

**Information Retained -** These are the essential fields that are kept to represent products and detect user intent for recommendations:

- **`product_name`** - Core product identity, essential for semantic search and user understanding.
- **`description`** - Rich semantic content containing features, specifications, and detailed product information that will be very useful in generation stage as context of the generator.
- **`product_category_tree`** - Hierarchical categorization is useful for category-based retrival of product.
- **`brand`** - Important for brand-specific searches and user preferences (for example 'nike shoes')
- **`discounted_price`** - Critical for budget-conscious recommendations and price-aware filtering (i also introduced price_category feature using this discounted price)
- **`pid`** - Unique identifier retained for product tracking and reference purposes

**Information Dropped**
These fields were dropped either because they don't matter to end users or have sparse/incomplete data:

- **`product_url`, `crawl_timestamp`** - Contain no semantic information relevant to product recommendations
- **`retail_price`** - Users primarily focus on discounted/final price rather than original pricing
- **`is_FK_Advantage_product`** - Platform-specific metadata that doesn't contribute to semantic understanding
- **`product_rating`, `overall_rating`** - Largely missing from dataset (mostly "No rating available"), unsuitable for analysis
- **`product_specifications`** - Contains complex nested JSON data requiring separate processing pipelines, beyond scope of text-focused RAG implementation
- **`image`** - Not utilizing visual features in this text-based recommendation system

Dropping information is computationally good for the encoding stage because the more information we use, the higher computation power is required at the encoding stage. 

### Q2.b.In the retrieval, did you use any textual retrieval algorithm? Can you explain why you should and/or should not? (2)

**No, I did not use traditional textual retrieval algorithms** like TF-IDF or BM25. Instead, I used a **semantic vector similarity approach** using FAISS (Facebook AI Similarity Search) with sentence transformer embeddings.

This is because the **textual algorithms rely on exact keyword matching and term frequency statistics** and completely fail when users are using different types of vocabulary in their queries that are not exactly matching the product description. Also, these algorithms fail to get any semantical understanding. For example, "waterproof" and "waterressistant" are treated as two exact different words but in reality, people often use them interchangably (though technically they are not the same).

Since we are using the FAISS alogn with embedding model `all-mpnet-base-v2`, we could capture semantic meaning, understand synonyms (e.g., budget == affordable == cheap) and thus handel quries and description that are writen in a different but related terminology. For the same reason, it could better align with practical search patterns of user and product characterisitcs.

The traditional algorithms are still useful in usecases where we need `100%` accuracy (for example law, medicine).

### Q2.c. Why do we need a model to embed? Should you use the same model for embedding documents and generation? Why or why not? (2)

**Need of the Embedding Models:** In RAG systems, we need embedding model to trainsform the vast amount of text into vector representation in such a way that captures the semantic meaning of the text. Here, similar texts (contents) tend to cluster together and we can leverage mathematical similarity calculation using vector operations (e.g., cosine similarity) and acheive semantic search on our texts. Also, there is the facility of converting varaible lenght text into fixed sized vectors that is efficient for indexing and processing in the RAG system. 

**Using Different Models for Embedding and Generation:** **Yes, we should use different specialized models.** This is because embeding models (encoder) are actually better at creating numerical representation of texts that can later be efficientlu compared for similarity. The SOTA models in `Sentence Transformers` are indeed fine tuned for efficient and maximum semantic similarity.
 
For Generation, we want our text to be as human-like as possible and maintain coherency. This is where decoder only models perform very well and they are pptimized on next-token prediction and instruction following tasks. SOTA models also demonstrate significant language reasoning, and creativity that makes them ideal for generation. As per our need, we also have the flexibility to fine tune generators to fit our application domain.

Due to such seperation, we also have the flexbility to experiment with different combination and check for optimal performance of RAG systems.

### Q2.d. Did you face any problem using the LLAMA model? If yes, what measure did you take to use the model? If not, why do you think you did not? (2)

I used the locally running llama3 model using ollama. I did not actually face any issue because **I was already using this for my recomendor system project where I use llama3 to extract entities using zero shot prompting. I have the boilerplates ready for local ollama client connection.**

I also did not have any major issues on computational restrictions (RAM:16GB, GPU: RTX3060) and each generation took no more than 3 seconds (due to GPU acceleration). 

If I had any resource restriction, maybe I would have chosen 3B/4B version of the model. I also saw that ollama supports quantized models (`llama3:8b-q4_K_M`) which works on lower end computer (but it would have compromised my quality of generation).

### Q2.e. Language models have limitation of context size. What is the context size of the model you use? Could you meet the requirement? (2)

The **maximum context length for Meta Llama 3 is 8,000 tokens.** This is equivalent to **roughly 6,000 to 8,000 words,** depending on the average word-to-token ratio [ref](https://www.ibm.com/think/topics/context-window) 

* During data preprocessing, I analyzed the maximum document length in my combined approach (the largest document type):
    - **Maximum document length**: **5,393 characters** (should be approximately `1200` tokens)
    - **Typical document length**: Much shorter, as this represents the longest outlier
    - **Product numbers**: Since we are ranking only 3 products, this means less context spent here as well.

* My Context Usage Analysis:
    - **System prompt**: ~200 tokens
    - **User query**: ~20 tokens  
    - **Retrieved products** (3 products, combined approach): ~800-1,200 tokens
    - **Instruction template**: ~200 tokens
    - **Response generation**: ~200-1,000 tokens
    - **Total estimated usage**: ~1,360-2,570 tokens

Thus, I could easily meet the requirement of context lenght. But if I have to provide any examples to the generator to improve my generation, I can not do it. In that case, I would consider truncating less important texts, for example, product description is the longest information and we may truncate it after 250-300 characters and it will still give use a good ammount of context. We can further reduce the context using efficient prompt designing as well.

### Q2.f. Can you suggest how we can personalize the recommendation for each user using RAG? (2)

To answer this question in the context of our flipcart dataset, we first need to see what we are actually given
`uniq_id, crawl_timestamp, product_url, product_name, product_category_tree, pid, retail_price, discounted_price, image, is_FK_Advantage_product, description, product_rating, overall_rating, brand, product_specifications` there is no information that can help us to undersrand users' personal preference or purchase behaviors. So in the context of our assignment, we can have no personalization. However, for personalizing recomendation for each user using RAG in other application scenario, we could consider the following (assuming it is possible in the given syste)

**User Profile Integration:** Injecting some user personal information will definately help us in the following manner.
- Here we can imagine user embedding as accumulation of their past interactions, browsing, demographics, location etc..
- Weight recent interactions higher than older ones using temporal decay functions
- Using profile intergration will also tackel **cold start problem** by leveraging user profile information for initial sessions.

**User History Injection:**
- We may include recent purchases/views in RAG context: "Based on your recent interest in [smartphones, electronics], here are recommendations..."
- We can maintain conversation context by Referencing previous queries within the same session.

**Behavioral Pattern Modeling:**
- We may adjust recommendations based on user's typical spending patterns (so users who spend less will not be recomended premium products, there by their recomended list becomes more effective and personalized)

Apart from these, there are some more advanced options like personalizing RAG prompts and dynamic context adaptation which require much more complexity and computational capability.

**Note:** While implementing such personalization, we would also need to be careful to ask user for consent.

### Q2.g. In this assignment, no performance evaluation was not asked. However, to evaluate the performance, how would you approach? (2)

Since it is a RAG system, we need to asses different components of the system, i) Encoder, ii) retrival and iii) Generator.

If I were asked to evaluatr the recomender system for this assignment, I would have first tried evaluating the component i and ii toghether using retrival quality. I do not think seperate evaluation for encoder and retrival will be necessary because their collective task is to retrive relevant products. For evaluating this retrived products, we would generally consider different ranking metrics like `precision@k`, `recall@k`, `NDCG@K`, `MRR`. But the issue is we do not have any ground truth for such calculation in our flipcart dataset. For example, when our user asks for "budget phone", we do not know which retrieved products are actually good recommendations. So instead of using the metrics, we have to manually evaluate the retrival with our intuition for now (cosine similarity for the retrived product does give us slight idea of the retrival quality, but it is not something definitive).

After that, I would check the generation quality manual checking first such as facutality of the generation. For example, when asking for a mobile, our retrivar gives a Huwaei router as 1st recomendation and if the generator is fooled by this and recomends it as a good phone, I would know that generation is not good. Also, I would look for coherence and relevance of the generator. So at the end of the day, we can see it comes down to human evaluation. Apart from that, there is an opportunity to use another LLM as a judge to automatically check for the quality of generation (Wang, 2024).

**Ref** <br>
Wang, Y., Hernandez, A. G., Kyslyi, R., & Kersting, N. (2024). Evaluating quality of answers for retrieval-augmented generation: A strong llm is all you need. arXiv preprint arXiv:2406.18064.